In [27]:
# Your code here
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a webpage
import pandas as pd
import numpy as np

In [2]:
#this url contains the data you need to scrape
url = "https://en.wikipedia.org/wiki/List_of_municipalities_in_Brazil_by_population"

In [7]:
#Define your User-Agent (important for Wikipedia to identify you)
USER_AGENT = "MyEducationalBot/1.0 (contact: syn@aungthuya.dev)"
headers = {'User-Agent': USER_AGENT}

In [8]:
#Scrape 
data  = requests.get(url, headers=headers).text 

In [9]:
#Create a Soup Object
soup = BeautifulSoup(data,"html.parser")

In [25]:
def clean_text(cell):
    text = cell.text
    text = text.replace('\n', '').replace(',', '')
    return text.strip()

all_tables = soup.find_all('table')
if len(all_tables) >= 2:
    second_table = all_tables[1]
else:
    print("Fewer than two tables found on the page.")
    
data = []
for tr in second_table.find_all('tr'):
    cells = tr.find_all('td')
    if (len(cells) < 4):
        continue
    data.append((
        clean_text(cells[0]), 
        clean_text(cells[1]),
        clean_text(cells[3])
    ))

In [28]:
df = pd.DataFrame(data, columns=["city", "state", "population"])
df.head()

,city,state,population
0,São Paulo,São Paulo,11451245
1,Rio de Janeiro,Rio de Janeiro,6211423
2,Brasília,Distrito Federal,2817068
3,Fortaleza,Ceará,2428678
4,Salvador,Bahia,2418005


In [46]:
#Create a LookUP table for States
states_lookup = df[['state']].copy()
states_lookup.drop_duplicates(inplace = True)
states_lookup.shape

(27, 1)

In [58]:

# 1. Split the text into words
# 2. Loop through each word and take the first character word[0]
# 3. Capitalize it using .upper()
# 4. Join them together with no space
states_lookup['acronym'] = states_lookup['state'].apply(
    lambda x: "".join([word[0].upper() for word in str(x).split()])
)

states_lookup.sort_values("state")

,index,state,acronym
25,68,Acre,A
13,15,Alagoas,A
23,48,Amapá,A
6,6,Amazonas,A
4,4,Bahia,B
3,3,Ceará,C
2,2,Distrito Federal,DF
21,38,Espírito Santo,ES
9,9,Goiás,G
12,14,Maranhão,M


I'll save the current state of LookUp table to csv file, and manually edit it to match with 
other items.

In [59]:
states_lookup.to_csv('DataSet/state_lookup.csv', index=False)